# Визуализация сезонных колебаний

На [странице](http://berkeleyearth.lbl.gov/stations/169045), откуда взят источник данных об истории измерений в метеостанции Ростова-на-Дону, предложены классические графики с динамикой колебаний вдоль одной оси времени.

Так как в дате содержится и год и месяц - мы можем визуализировать временной ряд в двух временных осях без вычисления каких-либо моделей.

In [ ]:
%pylab inline
import pandas as pd
import seaborn as sns

Поскольку информация о колонках указана в комментариях, то мы при импорте данных в таблицу указываем параметры
- без заголовков ( `header=None,` )
- свой перечень кратких имен (`names=[...]`)

In [ ]:
#                   Raw Data           QC    Continuity     Adjusted Data      Regional Expectation
# Year, Month, Temperature, Anomaly, Failed,   Breaks,   Temperature, Anomaly, Temperature, Anomaly

# u='http://berkeleyearth.lbl.gov/auto/Stations/TAVG/Text/169045-TAVG-Data.txt'
u='169045-TAVG-Data.txt'
D = pd.read_csv(u, delim_whitespace=' ',  comment='%', header=None,
                 names='year month v0 dv0 q br v dv reg dreg'.split())
D.tail()

Как мы видим, данные обрываются на дате 2013-11. В 19-м веке сбор данных также был нерегулярным. Поэтому для целей визуализации отберем колонку "Выверенные температурные данные" за 100 лет.

In [ ]:
X = D.loc[D.year>1912, ['year','month','v']]
X.head()

Готовим названия месяцев (для красоты)

In [ ]:
locale.LC_TIME

In [ ]:
# так можно получить готовый список 3-буквенных названий месяцев на английском
# momo = pd.tseries.frequencies.MONTHS
# momo = list(map(str.capitalize, momo))

# а так мы получим локализованные названия
import locale
locale.setlocale(locale.LC_TIME, '')
momo=[datetime.datetime(2016,i,1).strftime('%b') for i in arange(1,13)]
print(' '.join(momo))

Группируя по году и месяцу, мы создаем двойной индекс, который методом `unstack()` разделяем на индекс строк и индекс колонок, получая таким образом широкую таблицу.

In [ ]:
W = X[["year", "month", "v"]].groupby(by=["year", "month"]).max().unstack()
W.columns=momo
W.head()

In [ ]:
figure(figsize=(12, 8))
sns.heatmap(W, yticklabels=8, cmap="coolwarm")
xlabel('месяц'); ylabel('год'); title('Среднемесячная температура'); 

Обратите внимание:
- параметр `yticklabels` задает интервал между отметками по оси, чтобы метки не наезжали друг на друга
- дивергентная цветовая карта позволяет сосредоточиться на особо жарких и холодных месяцах.
- зимой 1941-го и во время оккупации данные не собирались

Если у нас стоит задача заполнить недостающие данные или спрогнозировать колебания в будущем, то в условиях стабильных предсказуемых циклов, какими являются сезонными, хороший результат дают средние значения для той же фазы цикла. 

В данном наборе данных для прогнозирования температуры в августе в будущем году или заполнения пропуска в 1941 году можно взять среднюю температуру в августе.

## Оценка аномалий

В наборе данных есть колонка, показывающая насколько текущее значение отклоняется от ожидаемого, т.е. показывает аномальность температуры в этом месяце.

In [ ]:
figure(figsize=(12, 8))
W_anomal = D.loc[D.year>1912,["year", "month", "dv"]].groupby(by=["year", "month"]).max().unstack() #делаем широкую таблицу
W_anomal.columns=momo
sns.heatmap(W_anomal, yticklabels=8, cmap="coolwarm")
xlabel('месяц'); ylabel('год'); title('Аномалия в температуре'); 

Сравним аномальность из таблицы с отклонением значений от среднего.

In [ ]:
W_dev = W - W.mean()
sns.heatmap(W_dev, yticklabels=8, cmap="coolwarm")
xlabel('месяц'); ylabel('год'); title('Отклонение от среднего'); 

По идентичности рисунков становится понятно, что "аномальность" именно так и рассчитали. Небольшие расхождения в нашем примере связаны с тем, что мы взяли среднее не за всю историю наблюдений, а за 100 лет.

> Обратите внимание, что в зимние месяцы температура нестабильна из года в год.

In [ ]:
W.std().plot(kind='bar', title='Вариативность среднемесячных температур в течение года');

В данном блокноте продемонстрированы приемы:
- двумерной цветовой визуализации
- создания и настройки подписей осей
- загрузки данных из сети с переопределением имен переменных